In [433]:
import sys
sys.path.append("..")

import datetime as dt
from ipywidgets import interact
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import statsmodels.api as sm
import tqdm
import vectorbtpro as vbt

import legitindicators as li
import pandas_ta as pta

from lib import bitget_loader, utils, indicators

# Setup

In [565]:
symbol = 'GALAUSDT'
freq = '4h'

is_start = dt.date(2021,1,1)
is_end = dt.date(2022,12,31)

os_start = dt.date(2023,1,1)
os_end = dt.date(2024,10,10)

df = pd.read_parquet(f"../data/candles/{symbol}.pq")

In [566]:
df = df.resample(freq).agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum', 'quoteVolume':'sum', 'nTrades':'sum', 'upVolume':'sum', 'upQuoteVolume':'sum'})

In [567]:
df['volume'] = df['volume'].replace({0:np.nan})
df['quoteVolume'] = df['quoteVolume'].replace({0:np.nan})
df['nTrades'] = df['nTrades'].replace({0:np.nan})
df['upVolume'] = df['upVolume'].replace({0:np.nan})
df['upQuoteVolume'] = df['upQuoteVolume'].replace({0:np.nan})

# Target

In [568]:
df['y_return'] = np.where(df['close']-df['open']>0, 1, np.where(df['close']-df['open']<0, -1, np.nan))
#df['y_return'] = df['close'].diff(5)>0
df['y_return'] = df['y_return'].shift(-1)
df = df.iloc[:-1].copy() # last row has no target, so drop it

In [569]:
# df['tma1'] = df['close'].rolling(12).mean().rolling(12).mean().shift(-11)
# df['tma2'] = df['close'].rolling(24).mean().rolling(24).mean().shift(-23)
# df['tma3'] = df['close'].rolling(48).mean().rolling(48).mean().shift(-47)

# df['y_return'] = np.where((df['tma1'].diff()>0) & (df['tma2'].diff()>0) & (df['tma3'].diff()>0), 1, np.where((df['tma1'].diff()<0) & (df['tma2'].diff()<0) & (df['tma3'].diff()<0), -1, np.nan))
# df['y_return'] = df['y_return'].shift(-1)
# df = df.iloc[:-1].copy() # last row has no target, so drop it

# Features

## Momentum

In [570]:
df['x_return_zs'] = pta.zscore(df['close'].pct_change(), 200)

In [571]:
for l in [7, 25, 99]:
    df[f'x_momnom_{l}'] = li.momentum_normalized(df['close'], l)
    df[f'sma_{l}'] = df['close'].rolling(l).mean()
    # df[f'x_sma_{l}_roc'] = df[f'sma_{l}'].pct_change()
    # df[f'x_sma_{l}_distance'] = (df['close']-df[f'sma_{l}'])/df[f'sma_{l}']

for p in itertools.combinations([7, 25, 99], 2):
    df[f'x_sma_{p[0]}_{p[1]}_distance'] = (df[f'sma_{p[0]}']-df[f'sma_{p[1]}'])/df[f'sma_{p[1]}']

In [572]:
for l in [7, 25, 99]:
    df[[f'x_adx_{l}', f'x_dmp_{l}', f'x_dmn_{l}']] = pta.adx(df['high'], df['low'], df['close'], l)
    df[f'x_adx_{l}'] = li.szladx(df[['high', 'low', 'close']].values, l)

## Volume

In [573]:
df['logvolume'] = np.log(df['volume'])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
df['volume'][df['volume'].rank(pct=True)<0.9].hist(ax=ax[0])
df['logvolume'].hist(ax=ax[1])
ax[0].set_title("Distribution of Volume")
ax[1].set_title("Distribution of log(Volume)")

In [575]:
df['x_logvolume_zs'] = pta.zscore(df['logvolume'], 200)
df['x_logvolume_zs_ma'] = df['x_logvolume_zs'].rolling(20).mean()
df['x_relative_volume_zs'] = df['logvolume'].groupby(df.index.time).apply(lambda d: pta.zscore(d, 40)).droplevel(0).sort_index()
df['x_relative_volume_zs_ma'] = df['x_relative_volume_zs'].rolling(20).mean()
df['x_volume_corr'] = df['volume'].rolling(20).corr(df['close'].pct_change().abs())

In [576]:
df['logticks'] = np.log(df['nTrades'])
df['x_logticks_zs'] = pta.zscore(df['logticks'], 200)
df['x_logticks_zs_ma'] = df['x_logticks_zs'].rolling(20).mean()
df['x_relative_ticks_zs'] = df['logticks'].groupby(df.index.time).apply(lambda d: pta.zscore(d, 40)).droplevel(0).sort_index()
df['x_relative_ticks_zs_ma'] = df['x_relative_ticks_zs'].rolling(20).mean()
df['x_ticks_corr'] = df['nTrades'].rolling(20).corr(df['close'].pct_change().abs())

## Volatility

In [577]:
df['tr'] = pta.true_range(df['high'], df['low'], df['close'])
df['logtr'] = np.log(df['tr'])

# df['x_tr_zs'] = pta.zscore(df['logtr'], 20)
# df['x_tr_zs_ma'] = df['x_tr_zs'].rolling(20).mean()
# df['x_relative_tr_zs'] = df['logtr'].groupby(df.index.time).apply(lambda d: pta.zscore(d, 20)).droplevel(0).sort_index()
# df['x_relative_tr_zs_ma'] = df['x_relative_tr_zs'].rolling(20).mean()
df['x_range_zs'] = pta.zscore(np.log((df['high']-df['low'])/df['open']), 200)

In [578]:
df['natr'] = df['tr'].ewm(20).mean()/df['close']

## Delta

In [579]:
df['volume_delta'] = 2*df['upVolume'] - df['volume']
for l in [1, 4, 12]:
    df[f'x_volume_delta_{l}_zs'] = pta.zscore(df['volume_delta'].rolling(l).sum(), 200)

## Test

In [580]:
df['x_dvolm'] = li.damiani_volatmeter(df[['open', 'high', 'low', 'close']].values, 13, 20, 40, 100, 1.4)
df['x_quote_vol_per_trade'] = pta.zscore(np.log(df['volume']/df['nTrades']), 200)
df['x_vol_per_trade'] = pta.zscore(np.log(df['quoteVolume']/df['nTrades']), 200)

In [581]:
# df['x_ebsw'] = li.ebsw(df['close'], 40, 10)
# df['x_tf'] = indicators.TrendFlex(df['close'], 30)
# df['x_rf'] = indicators.Reflex(df['close'], 30)
# df['x_voss'] = li.voss(df['close'].values, 20, 3, 0.25)

In [582]:
# for l in [9, 25, 99]:
#     df[f'x_ker_{l}'] = li.kaufman_er(df['close'], l)

## Plot

In [ ]:
fig = go.FigureWidget(make_subplots(rows=3, cols=1, shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]))
fig.add_trace(go.Candlestick(), row=1, col=1)
fig.add_trace(go.Scatter(), row=2, col=1)
fig.add_trace(go.Scatter(), row=3, col=1)
fig.update_layout(height=600, margin=dict(l=20,r=20,b=20,t=20), xaxis=dict(rangeslider=dict(visible=False)))

@interact(date=np.unique(df.index.date), col=df.columns, col2=df.columns)
def update(date, col, col2):
   with fig.batch_update():
      _sdf = df.loc[str(date)]
      fig.data[0].x, fig.data[0].open, fig.data[0].high = _sdf.index, _sdf['open'], _sdf['high']
      fig.data[0].low, fig.data[0].close = _sdf['low'], _sdf['close']
      fig.data[1].x, fig.data[1].y = _sdf.index, _sdf[col]
      fig.data[2].x, fig.data[2].y = _sdf.index, _sdf[col2]
      fig.update_layout()
fig


# Training

In [584]:
from pycaret.classification import ClassificationExperiment

In [585]:
ymask = ~pd.isna(df['y_return'])

x_train = df[ymask].loc[:is_end][utils.get_prefixed_cols(df, 'x_')].replace({np.inf:np.nan, -np.inf:np.nan})
y_train = df[ymask].loc[:is_end]['y_return']

In [ ]:
exp = ClassificationExperiment()
exp.setup(
    data=x_train, target=y_train,
    train_size=0.7,
    data_split_shuffle=False,
    data_split_stratify=False,
    numeric_imputation='mean',
    # remove_multicollinearity=True,
    # multicollinearity_threshold=0.8,
    normalize=False,
    pca=False,
    # feature_selection=True,
    # n_features_to_select=0.5,
    remove_outliers=False,
    fold_strategy='kfold',
    fold=5,
    fold_shuffle=False,
    # keep_features = x_cols,
    )

In [ ]:
exp.X_transformed.columns

In [ ]:
best = exp.compare_models(n_select=3, cross_validation=False)

In [ ]:
best = exp.create_model('gbc', cross_validation=False)
# best = exp.create_model('lr', cross_validation=False)

In [590]:
#exp.tune_model(best)

In [591]:
# exp.plot_model(best, 'threshold')

In [592]:
# exp.plot_model(best, plot='feature')

### Feature Importance

In [593]:
from sklearn.inspection import permutation_importance

In [594]:
# r = permutation_importance(best, exp.X_test_transformed, exp.y_test, n_repeats=30)

In [595]:
# for i in r.importances_mean.argsort()[::-1]:
#     if r.importances_mean[i] - 1 * r.importances_std[i] > 0:
#         print(f"{best.feature_name_[i]:<8}"
#               f" {r.importances_mean[i]:.3f}"
#               f" +/- {r.importances_std[i]:.3f}")

## Modelling Holdout Accuracy

## Backtest in Modelling Holdout

In [ ]:
bdf = df.loc[exp.test.index]
bdf['prediction_label'] = exp.predict_model(best)['prediction_label']
bdf['prediction_score'] = exp.predict_model(best)['prediction_score']

In [600]:
le = bdf['prediction_label'] == 1
le &= bdf['prediction_score'] > 0.65

se = bdf['prediction_label'] == -1
se &= bdf['prediction_score'] > 0.65

le = utils.crossover(le, 0.5)
se = utils.crossover(se, 0.5)

pf = vbt.Portfolio.from_signals(
    bdf['close'], open=bdf['open'], high=bdf['high'], low=bdf['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=3,
    time_delta_format=0,
)

In [ ]:
pf.stats()

In [ ]:
pf.value.plot()

### Signal Half-Life

In [604]:
pf = vbt.Portfolio.from_signals(
    bdf['close'], open=bdf['open'], high=bdf['high'], low=bdf['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=vbt.Param(np.arange(2, 20)),
    time_delta_format=0,
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
pf.trades.win_rate.plot(ax=ax[0], title='Win Rate')
pf.trades.profit_factor.plot(ax=ax[1], title='Profit Factor')
ax[0].axhline(0.5, alpha=0.5)
ax[1].axhline(1, alpha=0.5)

In [607]:
best_td_stop = 10

### TPSL Opt, Pct Based

In [608]:
tpsl_mults = np.arange(0.005, 0.2, 0.005)
pf = vbt.Portfolio.from_signals(
    bdf['close'], open=bdf['open'], high=bdf['high'], low=bdf['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=best_td_stop,
    time_delta_format=0,
    sl_stop=vbt.Param(tpsl_mults),
    tp_stop=vbt.Param(tpsl_mults),
    slippage=0.0001,
)

In [609]:
stat_result = pf.trades.get_profit_factor().unstack()
# stat_result = pf.trades.win_rate.unstack()
stat_result = stat_result.sort_index().sort_index(axis=1)

In [ ]:
sns.heatmap(stat_result, annot=False)

### TPSL Opt, ATR Based

In [611]:
tpsl_mults = np.arange(0.5, 8, 0.5)
pf = vbt.Portfolio.from_signals(
    bdf['close'], open=bdf['open'], high=bdf['high'], low=bdf['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=best_td_stop,
    time_delta_format=0,
    sl_stop=vbt.Param([x*bdf['natr'] for x in tpsl_mults]),
    tp_stop=vbt.Param([x*bdf['natr'] for x in tpsl_mults]),
    slippage=0.0001,
)

In [612]:
stat_result = pf.trades.get_profit_factor().unstack()
stat_result.index = stat_result.index.str[7:].astype(int)
stat_result.columns = stat_result.columns.str[7:].astype(int)
stat_result = stat_result.sort_index().sort_index(axis=1)
stat_result.index = tpsl_mults
stat_result.columns = tpsl_mults

In [ ]:
sns.heatmap(stat_result, annot=False)

In [616]:
best_sl = 2.5
best_tp = 5

In [617]:
pf = vbt.Portfolio.from_signals(
    bdf['close'], open=bdf['open'], high=bdf['high'], low=bdf['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=best_td_stop,
    time_delta_format=0,
    sl_stop=best_sl*bdf['natr'],
    tp_stop=best_tp*bdf['natr'],
    #sl_stop=0.05,
    #tp_stop=0.1,
    slippage=0.0001,
)

In [ ]:
pf.stats()

In [ ]:
pf.value.plot()

# Backtest OOS

In [621]:
final_model = exp.finalize_model(best)
# final_model = exp.finalize_model(best, model_only=True)

In [622]:
os_df = df.sort_index().loc[os_start:]
os_df['prediction_label'] = final_model.predict(os_df[utils.get_prefixed_cols(os_df, 'x_')]).values
os_df['prediction_score'] = final_model.predict_proba(os_df[utils.get_prefixed_cols(os_df, 'x_')])[:,1]
os_df['prediction_score'] = np.where(os_df['prediction_label']==1, os_df['prediction_score'], 1-os_df['prediction_score'])

In [623]:
le = os_df['prediction_label'] == 1
le &= os_df['prediction_score'] > 0.65
se = os_df['prediction_label'] == -1
se &= os_df['prediction_score'] > 0.65

le = utils.crossover(le, 0.5)
se = utils.crossover(se, 0.5)

pf = vbt.Portfolio.from_signals(
    os_df['close'], open=os_df['open'], high=os_df['high'], low=os_df['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=best_td_stop,
    time_delta_format=0,
    sl_stop=best_sl*os_df['natr'],
    tp_stop=best_tp*os_df['natr'],
    slippage=0.0001,
)

In [ ]:
pf.stats()

In [ ]:
pf.value.plot()

In [627]:
records = pf.trades.records
records['dt'] = os_df.index[records['entry_idx']]
records['exit_dt'] = os_df.index[records['exit_idx']]
records['sl'] = best_sl*os_df['natr'].iloc[records['entry_idx']].values
records['realized_r'] = records['return']/records['sl']
records = records.set_index('dt')

In [ ]:
records['realized_r'].cumsum().vbt.plot().show(renderer='png')

# Rolling Weekly Train-Predict

In [629]:
df['weeknum'] = (df.index.weekday.diff() < 0).cumsum()

In [ ]:
training_window = 26

df['prediction_label'] = np.nan
df['prediction_score'] = np.nan

for week in tqdm.tqdm(range(training_window+2, df['weeknum'].max()+1)):
    train_df = df[df['weeknum'].between(week-training_window, week-1)]
    pred_df = df[df['weeknum']==week]
    
    x_train = train_df[utils.get_prefixed_cols(train_df, 'x_')]
    ymask = ~train_df['y_return'].isna()
    xmask = ~x_train.isna().any(axis=1)
    
    final_model.fit(x_train[ymask&xmask], train_df['y_return'][ymask&xmask])

    df['prediction_label'].update(pd.Series(final_model.predict(pred_df[utils.get_prefixed_cols(pred_df, 'x_')]).values, pred_df.index))
    df['prediction_score'].update(pd.Series(final_model.predict_proba(pred_df[utils.get_prefixed_cols(pred_df, 'x_')])[:,1], pred_df.index))

df['prediction_score'] = np.where(df['prediction_label']==1, df['prediction_score'], 1-df['prediction_score'])

In [559]:
le = df['prediction_label'] == 1
le &= df['prediction_score'] > 0.75
se = df['prediction_label'] == -1
se &= df['prediction_score'] > 0.75

le = utils.crossover(le, 0.5)
se = utils.crossover(se, 0.5)

pf = vbt.Portfolio.from_signals(
    df['close'], open=df['open'], high=df['high'], low=df['low'],
    entries=le, short_entries=se,
    freq=freq,
    td_stop=best_td_stop,
    time_delta_format=0,
    sl_stop=best_sl*df['natr'],
    tp_stop=best_tp*df['natr'],
    slippage=0.0001,
)

In [ ]:
pf.stats()

In [ ]:
pf.value.plot()

In [563]:
records = pf.trades.records
records['dt'] = df.index[records['entry_idx']]
records['exit_dt'] = df.index[records['exit_idx']]
records['sl'] = best_sl*df['natr'].iloc[records['entry_idx']].values
records['realized_r'] = records['return']/records['sl']
records = records.set_index('dt')

In [ ]:
records['realized_r'].cumsum().plot()
records[records['direction']==0]['realized_r'].cumsum().plot()
records[records['direction']==1]['realized_r'].cumsum().plot()

# LTF Entries

In [ ]:
import importlib
importlib.reload(indicators)

In [141]:
df1 = pd.read_parquet(f"../data/candles/{symbol}.pq")

In [142]:
df1['ssmooth'] = indicators.SuperSmoother(df1['close'], 10)
df1['decycler'] = indicators.Decycler(df1['close'], 20)

In [143]:
df1['ma1'] = indicators.SuperSmoother(df1['close'], 9)
df1['ma2'] = indicators.SuperSmoother(df1['close'], 25)
df1['ma3'] = indicators.SuperSmoother(df1['close'], 99)

In [144]:
df1['prediction_label'] = df['prediction_label'].shift(freq='14min')
df1['prediction_score'] = df['prediction_score'].shift(freq='14min')
df1['prediction_label'] = df1['prediction_label'].ffill()
df1['prediction_score'] = df1['prediction_score'].ffill()

In [145]:
espf_length = 20
espf_bw = 1
df1['espf'] = indicators.SuperPassband(df1['close'], espf_length-espf_bw, espf_length+espf_bw)
df1['rms'] = indicators.RMS(df1['espf'], espf_length)

In [146]:
# le = df1['ma1']>df1['ma2']
# # le &= df1['ma2']>df1['ma3']

# se = df1['ma1']<df1['ma2']
# # se &= df1['ma2']<df1['ma3']

In [147]:
df1['tf'] = indicators.TrendFlex(df1['close'], 30)
df1['rf'] = indicators.Reflex(df1['close'], 30)

In [148]:
# le = utils.crossover(df1['rf'], df1['tf'])
# se = utils.crossover(df1['tf'], df1['rf'])

le = df1['rf'] >df1['tf']
se = df1['tf'] <df1['rf']

lx = utils.crossover(df1['tf'], df1['rf'])
sx = utils.crossover(df1['rf'], df1['tf'])

In [149]:
rms_mult = 1
le = utils.crossover(df1['espf'], rms_mult*df1['rms'])
se = utils.crossover(-rms_mult*df1['rms'], df1['espf'])
sx = utils.crossover(df1['espf'], df1['rms'])
lx = utils.crossover(df1['rms'], df1['espf'])

# le = utils.crossover(df1['ssmooth'], df1['decycler'])
# se = utils.crossover(df1['decycler'], df1['ssmooth'])
# sx = utils.crossover(df1['ssmooth'], df1['decycler'])
# lx = utils.crossover(df1['decycler'], df1['ssmooth'])

# le = df1['rf'] >df1['tf']
# se = df1['tf'] <df1['rf']

# lx = utils.crossover(df1['tf'], df1['rf'])
# sx = utils.crossover(df1['rf'], df1['tf'])

le &= df1['prediction_label'] == 1
le &= df1['prediction_score'] > 0.7
se &= df1['prediction_label'] == -1
se &= df1['prediction_score'] > 0.7

le = utils.crossover(le, 0.5)
se = utils.crossover(se, 0.5)

pf = vbt.Portfolio.from_signals(
    df1['close'], open=df1['open'], high=df1['high'], low=df1['low'],
    entries=le, #exits=lx,
    short_entries=se, #short_exits=sx,
    freq='1min',
    td_stop=16,
    time_delta_format=0,
    sl_stop=0.02,
    # tp_stop=0.04,
    # tp_stop=5*df['natr'],
    # slippage=0.0001,
)

In [ ]:
pf.stats()

In [ ]:
pf.value.plot()

In [152]:
records = pf.trades.records
records['dt'] = df1.index[records['entry_idx']]
records['exit_dt'] = df1.index[records['exit_idx']]
records['sl'] = 0.02
# records['sl'] = 3*df['natr'].iloc[records['entry_idx']].values
records['realized_r'] = records['return']/records['sl']
records = records.set_index('dt')

In [ ]:
records['realized_r'].cumsum().plot()
records[records['direction']==0]['realized_r'].cumsum().plot(label='long')
records[records['direction']==1]['realized_r'].cumsum().plot(label='short')
plt.legend()

In [154]:
df1['entry'] = np.where(le|se, df1['espf'], np.nan)

In [ ]:
fig = go.FigureWidget(make_subplots(rows=3, cols=1, shared_xaxes=True))
fig.add_trace(go.Candlestick(), row=1, col=1)
fig.add_trace(go.Scatter(mode='markers'), row=3, col=1)
fig.add_trace(go.Scatter(mode='markers'), row=3, col=1)
fig.add_trace(go.Scatter(), row=2, col=1)
fig.add_trace(go.Scatter(), row=2, col=1)
fig.add_trace(go.Scatter(), row=2, col=1)
fig.add_trace(go.Scatter(mode='markers'), row=2, col=1)
fig.update_layout(height=600, margin=dict(l=20,r=20,b=20,t=20), xaxis=dict(rangeslider=dict(visible=False)))

@interact(date=np.unique(df1['entry'].dropna().index.date), col=df1.columns, col2=df1.columns)
def update(date, col, col2):
   with fig.batch_update():
      _sdf = df1.loc[str(date)]
      fig.data[0].x, fig.data[0].open, fig.data[0].high = _sdf.index, _sdf['open'], _sdf['high']
      fig.data[0].low, fig.data[0].close = _sdf['low'], _sdf['close']
      fig.data[1].x, fig.data[1].y = _sdf.index, _sdf[col]
      fig.data[2].x, fig.data[2].y = _sdf.index, _sdf[col2]
      fig.data[3].x, fig.data[3].y = _sdf.index, _sdf['espf']
      fig.data[4].x, fig.data[4].y = _sdf.index, rms_mult*_sdf['rms']
      fig.data[5].x, fig.data[5].y = _sdf.index, -rms_mult*_sdf['rms']
      fig.data[6].x, fig.data[6].y = _sdf.index, _sdf['entry']
      fig.update_layout()
fig

In [ ]:
fig = go.FigureWidget(make_subplots(rows=1, cols=1, shared_xaxes=True))
fig.add_trace(go.Candlestick(), row=1, col=1)
fig.add_trace(go.Scatter(), row=1, col=1)
fig.add_trace(go.Scatter(), row=1, col=1)
fig.add_trace(go.Scatter(mode='markers', marker=dict(symbol='x', size=12, color='green')), row=1, col=1)
fig.add_trace(go.Scatter(mode='markers', marker=dict(symbol='x', size=12, color='red')), row=1, col=1)
fig.update_layout(height=800, margin=dict(l=20,r=20,b=20,t=20), xaxis=dict(rangeslider=dict(visible=False)))

@interact(r=records.index, col=df.columns, col2=df.columns)
def update(r, col, col2):
   with fig.batch_update():
      _r = records.loc[r]
      _sdf = df1.loc[str(_r.name.date())]
      
      print(_r['return'])
      fig.data[0].x, fig.data[0].open, fig.data[0].high = _sdf.index, _sdf['open'], _sdf['high']
      fig.data[0].low, fig.data[0].close = _sdf['low'], _sdf['close']
    #   fig.data[1].x, fig.data[1].y = [_r.name, _r['exit_dt']], [_r['sl'],_r['sl']]
    #   fig.data[2].x, fig.data[2].y = [_r.name, _r['exit_dt']], [_r['tp'],_r['tp']]
      fig.data[3].x, fig.data[3].y = [_r.name], [_r['entry_price']]
      fig.data[4].x, fig.data[4].y = [_r['exit_dt']], [_r['exit_price']]
      fig.update_layout()
fig